In [8]:
import numpy as np
import pyswarms as ps
import scipy as sc
import random
from timeit import default_timer as timer
import pandas as pd

2020-08-05 18:06:30,615 - qiskit.aqua.components.optimizers.nlopts.nloptimizer - INFO - NLopt is not installed. Please install it to use these global optimizers.


In [13]:
a = np.identity(7, dtype='complex')
import pandas as pd
np.savetxt('fids.csv', a, delimiter=",")
data = pd.DataFrame().

In [4]:
data_train = pd.read_csv('mnist_train.csv')

In [14]:
data_train_x = data_train.iloc[:,1:].values

In [15]:
data_train_y = data_train.iloc[:,:1].values

In [16]:
data_train_y_r = data_train_y.reshape(1,60000)

In [17]:
data = []
for i in range(10):
    data.append(data_train_x[np.reshape(data_train_y == i, (60000,))])

In [ ]:
S3 = np.array(np.diag([-4.5, -3.5, -2.5, -1.5, -0.5, 0.5, 1.5, 2.5, 3.5, 4.5]), dtype='complex')
S3

In [5]:
#@jit(target="cpu")
#SU(n) operation takes in 
def SUn(params, n):
    params = np.append(params, 0)
    params = np.mod(params, 2*np.pi)
    params = np.array(params, dtype='complex')
    params = params.reshape(n,n)
    z = np.zeros(n**4, dtype='complex')
    y = np.zeros(n**4, dtype='complex')
    z = z.reshape(n,n,n,n)
    y = y.reshape(n,n,n,n)
    for i in range(n):
        for j in range(n):
            if(i==j):
                zr = np.zeros(n**2)
                zr = zr.reshape(n,n)
                z[i][i] = zr
                y[i][i] = zr
            else:    
                z[i][j][i][i] = 1
                z[i][j][j][j] = -1
                y[i][j][i][j] = -1j
                y[i][j][j][i] = 1j
    for i in range(n):
        for j in range(i):
            params[j][i] = params[j][i]/4
            params[i][j] = params[i][j]/2
    U = np.zeros(n**2, dtype='complex')
    U = U.reshape(n,n)
    d=n
    for m in range(d-1):
        for n1 in range(m+1, d):
            #if(m==n1 or params[m][n1] == 0 or params[n1][m] == 0):
            #    print('ghapla')
            U += 1j*params[n1][m]*z[m][n1] + 1j*params[m][n1]*y[m][n1]
    for l in range(d-1):
        #if(params[l][l] == 0 or l == d-1):
        #    print('ghapla')
        U += 1j*params[l][l]*z[l][d-1]
        #print(np.linalg.det(U))
    U = sc.linalg.expm(U)
    return U




In [19]:
@jit(parallel = True)
def prepare_density_matrices(params):
    root5 = np.sqrt(1/5)
    state_vec_init = [root5 for _ in range(5)]
    state_vec_init = np.array(state_vec_init, dtype='complex')
    S3 = np.array(np.diag([-2,-1,0,1,2]), dtype='complex')
    matrices = np.zeros(shape = (5,5,5), dtype = 'complex')
    
    for digit in data[:5]:
        rho_total = np.zeros(25, dtype='complex')
        rho_total = rho_total.reshape(5,5)
        counter = 0
        i=0
        for digit_data in digit[:2000]:
            opr = S3 * np.dot(params, digit_data)
            state_vec = np.dot(sc.linalg.expm(1j*opr), state_vec_init)
            state_vec = state_vec.reshape(5,1)
            state_vec_dag = state_vec.conj()
            state_vec_dag = state_vec.reshape(1,5)
            rho = np.dot(state_vec, state_vec_dag)
            rho_total += rho
            counter += 1
        rho_total /= counter
        matrices[i] = rho_total
        i += 1
    matrices = np.array(matrices, dtype='complex')
    return matrices






In [21]:
st = timer()
p = np.array([random.uniform(1,10) for _ in range(784)])
prepare_density_matrices(p)
en = timer()
print(en-st)

8.500543199999811


In [6]:
params = [random.uniform(0,2*np.pi) for _ in range(2499)]
start = timer()
U10 = SUn(params, 50)
end = timer()
print(end-start)

0.07428239999990183


In [12]:
np.linalg.det(U10)


(1.000000000000011-8.715250743307578e-15j)

In [40]:
@jit(target="cpu", parallel=True)
def forward_prop(params):
    params_state_prep = params[:784]
    params_SUn = params[784:]
    matrices = prepare_density_matrices(params_state_prep)
    SUn_opr = SUn(params_SUn, 5)
    cost = 0
    #print('done\n')
    for i in prange(5):
        matrix = matrices[i]
        SUn_opr_dag = SUn_opr.T
        SUn_opr_dag = SUn_opr_dag.conj()
        final_rho = np.matmul(SUn_opr, matrix)
        final_rho = np.matmul(final_rho, SUn_opr_dag)
        target_rho = np.zeros(25, dtype='complex')
        target_rho = target_rho.reshape(5,5)
        target_rho[i][i] = 1
        T = final_rho - target_rho
        cost += sc.linalg.norm(T)
    #print(cost)

    return cost




In [33]:
st = timer()
params = [random.uniform(0,2*np.pi) for _ in range(808)]
forward_prop(np.array(params))
en = timer()
print(en-st)

2020-07-31 22:35:46,237 - numba.transforms - INFO - finding looplift candidates
done

5.0937718696029295
8.861243299999842


In [38]:
def f(x):
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

<RootLogger root (INFO)>


In [42]:
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of GlobalBestPSO
optimizer = ps.single.GlobalBestPSO(n_particles=20, dimensions=808,
                                    options=options)

# Perform optimization
stats = optimizer.optimize(f, iters=100, n_processes=12)

2020-07-31 22:42:56,032 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


pyswarms.single.global_best:   0%|          |0/100